<a href="https://colab.research.google.com/github/endiesworld/2110ACDS_T7_C_Predict/blob/main/2110ACDS_T7_starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDSA Movie Recommendation 2022

© Explore Data Science Academy

---
### Honour Code

**2110ACDS_T6**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.


  

<h2><center> EDSA Movie Recommendation 2022</h2></center>
<figure>
<center><img src ="./assets/movies.png" width = "800" height = '500'/>

*Introduction*
<p align = "justify">Recommender System is a system that seeks to predict or filter preferences according to the user’s choices. Recommender systems are utilized in a variety of areas, and in this project we will use a recommender system to recommend movies for movie lovers.


*About the problem*
<p align = "justify">PUT PROBLEM STATEMENT HERE.

*Objective*
<p align = "justify"> We aim to provide an accurate and robust solution to this problem, by providing personalised recommendations to users of this product, and generating platform affinity for the streaming services which best facilitates their audience's viewing

*Process*
<p align = "justify"> In order to achieve this objective the team will follow the process below:-

1. analyse the supplied data, identify potential errors in the data and clean the existing data set;

2. determine if additional features can be added to enrich the data set;

3. build a model that is capable of predicting how a user will rate a movie;

4. evaluate the accuracy of the best machine learning model;

5. accurately predicting how a user will rate a movie they have not yet viewed, based on their historical preferences, and

6. explain the inner working of the model to a non-technical audience.

<a id="cont"></a>

## Table of Contents

<a href=#one>1. Importing Packages</a>

<a href=#two>2. Loading Data</a>

<a href=#three>3. Exploratory Data Analysis (EDA)</a>

<a href=#four>4. Data Engineering</a>

<a href=#five>5. Modeling</a>

<a href=#six>6. Model Performance</a>

<a href=#seven>7. Model Explanations</a>

 <a id="one"></a>
## 1. Importing Packages
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Importing Packages ⚡ |
| :--------------------------- |
| In this section you are required to import, and briefly discuss, the libraries that will be used throughout your analysis and modelling. |

---

In [2]:

# Import comet_ml at the top of your file
# from comet_ml import Experiment

# # Create an experiment with your api key
# experiment = Experiment(
#     api_key="emBEBYBp72gW5tfeZBSGftD0Y",
#     project_name="movie-recommendation",
#     workspace="emmanuelokoro",
#     log_code = True
# )

In [1]:
# Libraries for importing and loading data
import numpy as np
import pandas as pd
import scipy as sp # <-- The sister of Numpy, used in our code for numerical efficientcy. 
import matplotlib.pyplot as plt
import seaborn as sns


# Entity featurization and similarity computation
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn import preprocessing

# Libraries used during sorting procedures.
import operator # <-- Convienient item retrieval during iteration 
import heapq # <-- Efficient sorting of large lists
# Setting global constants to ensure notebook results are reproducible

from time import time

RANDOM_STATE = 42


import warnings
warnings.filterwarnings('ignore')

<a id="two"></a>
## 2. Loading the Data
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Loading the data ⚡ |
| :--------------------------- |
| In this section you are required to load the data from the `df_train` file into a DataFrame. |

---

### 2.1 Brief description of the data



In [8]:
# load the data
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
genome_scores = pd.read_csv('./data/genome_scores.csv')
genome_tags = pd.read_csv('./data/tags.csv')
imdb_data = pd.read_csv('./data/imdb_data.csv')
links = pd.read_csv('./data/links.csv')
movies = pd.read_csv('./data/movies.csv')
# tags = pd.read_csv('./data/tags.csv')

In [5]:
# Preview train dataset
print('The Shape of the data is: ', train.shape)
train.head()

The Shape of the data is:  (10000038, 4)


,userId,movieId,rating,timestamp
0,5163,57669,4.0,1518349992
1,106343,5,4.5,1206238739
2,146790,5459,5.0,1076215539
3,106362,32296,2.0,1423042565
4,9041,366,3.0,833375837


In [6]:
train['userId'].nunique()

162541

In [9]:
# Preview train dataset
print('The Shape of the data is: ', test.shape)
test.head()

The Shape of the data is:  (5000019, 2)


,userId,movieId
0,1,2011
1,1,4144
2,1,5767
3,1,6711
4,1,7318


In [8]:
test['movieId'].nunique()

39643

In [9]:
# Preview genome_scores dataset
print('The Shape of the data is: ', genome_scores.shape)
genome_scores.head()

The Shape of the data is:  (15584448, 3)


,movieId,tagId,relevance
0,1,1,0.02875
1,1,2,0.02375
2,1,3,0.06250
3,1,4,0.07575
4,1,5,0.14075


In [10]:
# Preview genome_scores dataset
print('The Shape of the data is: ', genome_tags.shape)
genome_tags.head()

The Shape of the data is:  (1093360, 4)


,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


In [11]:
# Preview imdb_data dataset
print('The Shape of the data is: ', imdb_data.shape)
imdb_data.head()

The Shape of the data is:  (27278, 6)


,movieId,title_cast,director,runtime,budget,plot_keywords
0,1,Tom Hanks|Tim Allen|Don Rickles|Jim Varney|Wal...,John Lasseter,81.0,"$30,000,000",toy|rivalry|cowboy|cgi animation
1,2,Robin Williams|Jonathan Hyde|Kirsten Dunst|Bra...,Jonathan Hensleigh,104.0,"$65,000,000",board game|adventurer|fight|game
2,3,Walter Matthau|Jack Lemmon|Sophia Loren|Ann-Ma...,Mark Steven Johnson,101.0,"$25,000,000",boat|lake|neighbor|rivalry
3,4,Whitney Houston|Angela Bassett|Loretta Devine|...,Terry McMillan,124.0,"$16,000,000",black american|husband wife relationship|betra...
4,5,Steve Martin|Diane Keaton|Martin Short|Kimberl...,Albert Hackett,106.0,"$30,000,000",fatherhood|doberman|dog|mansion


In [12]:
# Preview links dataset
print('The Shape of the data is: ', links.shape)
links.head()

The Shape of the data is:  (62423, 3)


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [13]:
# Preview movies dataset
print('The Shape of the data is: ', movies.shape)
movies.head()

The Shape of the data is:  (62423, 3)


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [14]:
movies.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [15]:
# Preview tags dataset
# print('The Shape of the data is: ', tags.shape)
# tags.head()

#### Dataset summary


<a id="three"></a>
## 3. Exploratory Data Analysis (EDA)
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Exploratory data analysis ⚡ |
| :--------------------------- |
| In this section, you are required to perform an in-depth analysis of all the variables in the DataFrame. |

---


### 3.1 Exploratory Data Analysis
*What is Exploratory data analysis?*
    Exploratory data analysis (EDA) is the process of analysing and investigating data sets and summarizing their main characteristics, often employing both non-graphical and graphical methods. 

*Why is conducting EDA important?*
    It aids in determining how best to manipulate data to get the required answers, expose trends, patterns, and relationships that are not readily apparent i.e. get insights into the dataset.

*How is EDA conducted?*
    EDA can be conducted in the following ways:
- **Univariate**:- \
    i. **non-graphical**:- This is simplest form of data analysis, where the data being analyzed consists of just one variable. Since it’s a single variable, it doesn’t deal with causes or relationships.\
    ii. **graphical**:- Non-graphical methods don’t provide a full picture of the data. Graphical methods are therefore required. It involves visual exploratory analysis of the data.
- Multivariate:-  \
    i. **non-graphical**:- Multivariate non-graphical EDA techniques generally show the relationship between two or more variables of the data through cross-tabulation or statistics. \
    ii. **graphical**:- Multivariate data uses graphics to display relationships between two or more sets of data. The most used graphic is a grouped bar plot or bar chart with each group representing one level of one of the variables and each bar within a group representing the levels of the other variable.
    
To achieve the above, while considering the volume of dataset for this project, we make use of this python module **pandas_profiling**

#### 3.1.1 pandas_profiling
Pandas profiling is an open source Python module with which we can quickly do an exploratory data analysis with just a few lines of code. It offers report generation for the dataset with lots of features and customizations for the report generated.

In [16]:
# Generate EDA report of train dataset
# from pandas_profiling import ProfileReport
# profile = ProfileReport(train, title="Report")
# profile


In [17]:
# Generate report for genome_scores
# profile = ProfileReport(genome_scores, title="genome_scores report")
# profile


#### summarize the above.

 **Descriptive Statistics**

>Descriptive statistics summarize the data by computing mean, median, mode, standard deviation likewise.descriptive statistics describe the dataset in a way simpler manner through;

*   The measure of central tendency 
>*  Mean:- The average value 
>*  Median:- The mid point value 
>*  Mode:- The most common value

*   Measure of spread  
>* Percentiles:- Percentiles are used in statistics to give you a number that describes the value that a given percent of the values are lower than.
>* standard deviation:-a number that describes how spread out the values are.
*  Measure of symmetry 
>* Skewness:- a measure of the asymmetry of the probability distribution of a real-valued random variable about its mean.
>>* If skewness is less than -1 or greater than 1, the distribution is highly skewed.
>>* If skewness is between -1 and -0.5 or between 0.5 and 1, the distribution is moderately skewed.
>>* If skewness is between -0.5 and 0.5, the distribution is approximately symmetric. 
*  Measure of Peakedness 
>* Kurtosis:-  a measure of relative peakedness of a probability distribution, or alternatively how heavy or how light its tails are. A standard normal distribution has kurtosis of 3 and is recognized as mesokurtic. An increased kurtosis (>3) can be visualized as a thin “bell” with a high peak whereas a decreased kurtosis corresponds to a broadening of the peak and “thickening” of the tails. Kurtosis >3 is recognized as leptokurtic and <3 as platykurtic (lepto=thin; platy=broad).
>>








In [18]:
# look at data statistics


### 3.2 Key Insights from EDA 


<a id="four"></a>
## 4. Data Engineering
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Data engineering ⚡ |
| :--------------------------- |
| In this section you are required to: clean the dataset, and possibly create new features - as identified in the EDA phase. |

---

Considering the large volume of dataset we have, we shall restrict this work to only userIds present in the test dataset.

**Unique userid**
We want to evaluate the difference between the unique userId in the Train dataset and Test dataset

In [10]:
test['userId'].nunique()

162350

In [11]:
test_case = test['userId'].nunique()
train_case = train['userId'].nunique()
print('The difference in unique userID count between train and test data set is:', (train_case - test_case))

The difference in unique userID count between train and test data set is: 191


From the above, proceed to extract these 191 userIds, that are not required for prediction

In [12]:
test_userids = test['userId'].unique().tolist()
test_userids[:20]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [13]:
#Extract rows with userId present in test userId
useful_train = train[train['userId'].isin(test_userids)]
useful_train.shape

(9997845, 4)

#### Dividing dataset into chunks

We shall now proceed to divide the **useful_train dataset** into **500** Chunks, to enable easily work with them.
**Steps**
- Order train dataset by userId
- Get the values of all userId into a list 
- Divide the list into 40 part
- Use the individual list gotten from above to separate your useful_train dataset into 20 chunks

In [14]:
# Order train dataset by userId
useful_train.sort_values(by=['userId'], inplace= True)
useful_train.head()

,userId,movieId,rating,timestamp
5122500,1,3949,5.0,1147868678
9153002,1,1175,3.5,1147868826
6923102,1,6016,5.0,1147869090
724395,1,7323,3.5,1147869119
2805472,1,4973,4.5,1147869080


In [15]:
# View the last five rows of the train data
useful_train.tail()

,userId,movieId,rating,timestamp
9103441,162541,2396,4.0,1240952712
547504,162541,4973,4.5,1240950790
7991803,162541,2539,1.0,1240950911
1861237,162541,1201,3.0,1240953800
9435687,162541,1230,3.5,1240951041


In [16]:
# Get the values of all userId into a list 
train_userids = useful_train['userId'].unique().tolist()
print(f'There are {len(train_userids)} of different userIds in the useful train dataset')

There are 162350 of different userIds in the useful train dataset


#### Order the test dataset by userId

In [17]:
# Order test dataset 
useful_test = test.sort_values(by=['userId'])
useful_test.head()

,userId,movieId
0,1,2011
1,1,4144
2,1,5767
3,1,6711
4,1,7318


In [38]:
useful_test.nunique()

userId     162350
movieId     39643
dtype: int64

From the above result, we proceed to divide 162350 by 40, which give an approximate value of 4059. Which means when we divide the useful train dataset into chuncks, we would have 4059 unique userIds in the first 39 chunks, and 4049 in the last chunks. 

In [28]:
# A function that generate a list of chunks
# def create_chunk_list(obj, limit, cycle):
#     """
#         This function accepts a list of data as obj argument, a limit value which is the maximum chunk size in each 
#         chunk and a cycle, which is the numbers/size of chunks to be created
#     """
#     chunks = []
#     start = 0
#     new_limit = limit
#     for i in range(cycle):
#         chunks.append(obj[start:new_limit])
#         start = start + limit
#         new_limit = new_limit + limit
        
#     return chunks
            

In [29]:
# Create 1624 chunks of userId, with a chunk size of 100
# userId_chunks = create_chunk_list(train_userids, 100, 1624)

# # Random evaluation of the userId chunk size
# print(f'We have {len(userId_chunks)} chunks of dataset')
# print(f'The length of the first chunk is: {len(userId_chunks[0])}')
# print(f'The length of the 1623 chunk is: {len(userId_chunks[1622])}')
# print(f'The length of the last chunk is: {len(userId_chunks[-1])}')

We separated the uniqueIds into 16234 chuncks which gives us 100 unique userIds in each of the first 1623 chunks and 
50 unique userId in the last chunk.
For proper understanding, let us view the first 10 userIds in three random chunks

In [30]:
# Random evaluation of the first 10 userids in the chunks created above

# print(f'The first 10 userIds in the first chunk are: {userId_chunks[0][:10]}')
# print(f'The first 10 userIds in the 1623 chunk is: {userId_chunks[1622][:10]}')
# print(f'The first 10 userIds in the last chunk is: {userId_chunks[-1][:10]}')

#### Divide dataset

For simplicity and explanatory reasons, we have chosen to divide the dataset manually i.e one line at a time. A faster will be to dynamically create it and store in a list. This operation will generate **train_chunk_1** to **train_chunk_1623**

In [31]:
# Divide useful trian into chunks, to create train_chunk_1 to train_chunk_40
# train_chunks_name = []
# for index, chunkId in enumerate(userId_chunks):
#     chunk_name = "train_chunk_{0}".format(index + 1)
#     globals()[chunk_name] = useful_train[useful_train['userId'].isin(chunkId)]
#     train_chunks_name.append(chunk_name)

In [32]:
#Test the above operation by printing the first five rows of the first chunk and the last five rows of the last chunk
# train_chunk_1.head()

In [33]:
# View the last five rows of the last chunk of the test data
# train_chunk_1624.tail()

In [34]:
# View the first and last chunk names created above
# print(f'The first chunk name created above is: {train_chunks_name[0]}')
# print(f'The last chunk name created above is: {train_chunks_name[-1]}')

#### Merging of tables

We proceed to merge tables with all train_chunks dataset created above. We execute this merge operation, using the userId as a reference. This operation will generate variables **merge_table_1** to **merge_table_40**

In [35]:
# Merge chunk with imdb_data 
# merge_chunks_name = []
# for index, chunk_name in enumerate(train_chunks_name):
#     merge_name = "merge_chunk_{0}".format(index + 1)
#     globals()[merge_name] = globals()[chunk_name].merge(imdb_data, on = 'movieId', how= 'left')
#     merge_chunks_name.append(merge_name)
    
# # View the result of the first five rows in the first merged chunks
# merge_chunk_1.head()

In [36]:
# Merge train data set with imdb_data tabl
useful_train = useful_train.merge(imdb_data, on = 'movieId', how= 'left')

# Merge test data set with imdb_data tabl
useful_test = test.merge(imdb_data, on = 'movieId', how= 'left')

In [37]:
# View the result of the last five rows in the last train table 
useful_train.tail()

,userId,movieId,rating,timestamp,title_cast,director,runtime,budget,plot_keywords
9997840,162541,2396,4.0,1240952712,Geoffrey Rush|Tom Wilkinson|Steven O'Donnell|T...,John Madden,123.0,"$25,000,000",william shakespeare character|shakespeare play...
9997841,162541,4973,4.5,1240950790,Audrey Tautou|Mathieu Kassovitz|Rufus|Lorella ...,Guillaume Laurant,122.0,"$10,000,000",female protagonist|paris france|france|montmar...
9997842,162541,2539,1.0,1240950911,Robert De Niro|Billy Crystal|Lisa Kudrow|Chazz...,Kenneth Lonergan,103.0,"$80,000,000",sex scene|mafia boss|mob boss|sexual intercourse
9997843,162541,1201,3.0,1240953800,NaN,NaN,NaN,NaN,NaN
9997844,162541,1230,3.5,1240951041,NaN,NaN,NaN,NaN,NaN


In [38]:
# View the result of the last five rows in the last test table 
useful_test.tail()

,userId,movieId,title_cast,director,runtime,budget,plot_keywords
5000014,162541,4079,NaN,NaN,NaN,NaN,NaN
5000015,162541,4467,NaN,NaN,NaN,NaN,NaN
5000016,162541,4980,Keanu Reeves|Alex Winter|William Sadler|Joss A...,Chris Matheson,93.0,"$20,000,000",lucifer morningstar character|death character|...
5000017,162541,5689,Dustin Hoffman|Nicole Kidman|Loren Dean|Bruce ...,E.L. Doctorow,106.0,"$48,000,000",waterfall|lake|female full frontal nudity|1930s
5000018,162541,7153,Noel Appleby|Ali Astin|Sean Astin|David Aston|...,J.R.R. Tolkien,201.0,"$94,000,000",orc|epic|battle|journey


In [39]:
# # Drop columns that are considered not necessary from the result of the merge operation above

# for  merge_name in merge_chunks_name:
#     globals()[merge_name] = globals()[merge_name].drop(columns=['timestamp', 'runtime', 'budget'])
    
# # View the result of the first five rows in the first merged chunks
# merge_chunk_1.head()

In [40]:
# Drop columns that are considered not necessary from the result of the merge operation above
useful_train = useful_train.drop(columns=['timestamp', 'runtime', 'budget'])
useful_test = useful_test.drop(columns=['runtime', 'budget'])

In [41]:
# Merge tables with movies table 

useful_train = useful_train.merge(movies, on = 'movieId', how= 'left')
    
useful_test = useful_test.merge(movies, on = 'movieId', how= 'left')

# View first five rows of newly merged table
useful_train.head()

,userId,movieId,rating,title_cast,director,plot_keywords,title,genres
0,1,3949,5.0,Ellen Burstyn|Jared Leto|Jennifer Connelly|Mar...,Hubert Selby Jr.,drug addiction|heroin|sex show|sex scene,Requiem for a Dream (2000),Drama
1,1,1175,3.5,Pascal Benezech|Dominique Pinon|Marie-Laure Do...,Jean-Pierre Jeunet,black comedy|absurd comedy|surrealist|bed,Delicatessen (1991),Comedy|Drama|Romance
2,1,6016,5.0,Alexandre Rodrigues|Leandro Firmino|Phellipe H...,Kátia Lund,photographer|slum|gang|brazil,City of God (Cidade de Deus) (2002),Action|Adventure|Crime|Drama|Thriller
3,1,7323,3.5,Daniel Brühl|Katrin Saß|Chulpan Khamatova|Mari...,Bernd Lichtenberg,coma|german democratic republic|capitalism|pol...,"Good bye, Lenin! (2003)",Comedy|Drama
4,1,4973,4.5,Audrey Tautou|Mathieu Kassovitz|Rufus|Lorella ...,Guillaume Laurant,female protagonist|paris france|france|montmar...,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Comedy|Romance


In [42]:
# View first five rows of newly merged table
useful_test.head()

,userId,movieId,title_cast,director,plot_keywords,title,genres
0,1,2011,NaN,NaN,NaN,Back to the Future Part II (1989),Adventure|Comedy|Sci-Fi
1,1,4144,Maggie Cheung|Tony Chiu-Wai Leung|Ping Lam Siu...,Kar-Wai Wong,adultery|loneliness|neighbor|infidelity,In the Mood For Love (Fa yeung nin wa) (2000),Drama|Romance
2,1,5767,NaN,NaN,NaN,Teddy Bear (Mis) (1981),Comedy|Crime
3,1,6711,Scarlett Johansson|Bill Murray|Akiko Takeshita...,Sofia Coppola,older man younger woman relationship|lonelines...,Lost in Translation (2003),Comedy|Drama|Romance
4,1,7318,Jim Caviezel|Maia Morgenstern|Christo Jivkov|F...,Benedict Fitzgerald,suffering|torture|brutality|whipping,"Passion of the Christ, The (2004)",Drama


#### Data formating

Before we can use any string vectorizer on our data, we need to properly format the data.

In [43]:
# fill NaN with empty string
useful_train = useful_train.fillna('')
useful_train.tail()

,userId,movieId,rating,title_cast,director,plot_keywords,title,genres
9997840,162541,2396,4.0,Geoffrey Rush|Tom Wilkinson|Steven O'Donnell|T...,John Madden,william shakespeare character|shakespeare play...,Shakespeare in Love (1998),Comedy|Drama|Romance
9997841,162541,4973,4.5,Audrey Tautou|Mathieu Kassovitz|Rufus|Lorella ...,Guillaume Laurant,female protagonist|paris france|france|montmar...,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Comedy|Romance
9997842,162541,2539,1.0,Robert De Niro|Billy Crystal|Lisa Kudrow|Chazz...,Kenneth Lonergan,sex scene|mafia boss|mob boss|sexual intercourse,Analyze This (1999),Comedy
9997843,162541,1201,3.0,,,,"Good, the Bad and the Ugly, The (Buono, il bru...",Action|Adventure|Western
9997844,162541,1230,3.5,,,,Annie Hall (1977),Comedy|Romance


In [44]:
# fill NaN with empty string
useful_test = useful_test.fillna('')
useful_test.head()

,userId,movieId,title_cast,director,plot_keywords,title,genres
0,1,2011,,,,Back to the Future Part II (1989),Adventure|Comedy|Sci-Fi
1,1,4144,Maggie Cheung|Tony Chiu-Wai Leung|Ping Lam Siu...,Kar-Wai Wong,adultery|loneliness|neighbor|infidelity,In the Mood For Love (Fa yeung nin wa) (2000),Drama|Romance
2,1,5767,,,,Teddy Bear (Mis) (1981),Comedy|Crime
3,1,6711,Scarlett Johansson|Bill Murray|Akiko Takeshita...,Sofia Coppola,older man younger woman relationship|lonelines...,Lost in Translation (2003),Comedy|Drama|Romance
4,1,7318,Jim Caviezel|Maia Morgenstern|Christo Jivkov|F...,Benedict Fitzgerald,suffering|torture|brutality|whipping,"Passion of the Christ, The (2004)",Drama


In [45]:
# Remove delimeters(Separators) from string data
def splitter(df, col_list, delim):
    """
        This function accepts a dataframe(df) and a list of columns(col_list), which contains the delimiter
        to be removed, it also accepts the delimiter which is to be removed
    """
    new_df = df.copy()
    
    for col in col_list:
        new_df[col] = new_df[col].str.split(delim).str.join(' ')
    
    return new_df
        

In [46]:
# Remove separators form string data

useful_train = splitter(useful_train, ['title_cast', 'director', 'plot_keywords', 'genres'], '|')
    
useful_test = splitter(useful_test, ['title_cast', 'director', 'plot_keywords', 'genres'], '|')
# View the result of the first five rows in the first merged chunks

useful_train.head()

,userId,movieId,rating,title_cast,director,plot_keywords,title,genres
0,1,3949,5.0,Ellen Burstyn Jared Leto Jennifer Connelly Mar...,Hubert Selby Jr.,drug addiction heroin sex show sex scene,Requiem for a Dream (2000),Drama
1,1,1175,3.5,Pascal Benezech Dominique Pinon Marie-Laure Do...,Jean-Pierre Jeunet,black comedy absurd comedy surrealist bed,Delicatessen (1991),Comedy Drama Romance
2,1,6016,5.0,Alexandre Rodrigues Leandro Firmino Phellipe H...,Kátia Lund,photographer slum gang brazil,City of God (Cidade de Deus) (2002),Action Adventure Crime Drama Thriller
3,1,7323,3.5,Daniel Brühl Katrin Saß Chulpan Khamatova Mari...,Bernd Lichtenberg,coma german democratic republic capitalism pol...,"Good bye, Lenin! (2003)",Comedy Drama
4,1,4973,4.5,Audrey Tautou Mathieu Kassovitz Rufus Lorella ...,Guillaume Laurant,female protagonist paris france france montmar...,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Comedy Romance


In [47]:
# Merge interested columns values for vectorization

useful_train['key_words'] = (pd.Series(useful_train[['title_cast', 'director', 'plot_keywords', 'genres']].fillna('')
                      .values.tolist()).str.join(' '))
   

useful_test['key_words'] = (pd.Series(useful_test[['title_cast', 'director','plot_keywords', 'genres']].fillna('')
                      .values.tolist()).str.join(' '))
# View the result of the first five rows in the first merged chunks
useful_train.head()

,userId,movieId,rating,title_cast,director,plot_keywords,title,genres,key_words
0,1,3949,5.0,Ellen Burstyn Jared Leto Jennifer Connelly Mar...,Hubert Selby Jr.,drug addiction heroin sex show sex scene,Requiem for a Dream (2000),Drama,Ellen Burstyn Jared Leto Jennifer Connelly Mar...
1,1,1175,3.5,Pascal Benezech Dominique Pinon Marie-Laure Do...,Jean-Pierre Jeunet,black comedy absurd comedy surrealist bed,Delicatessen (1991),Comedy Drama Romance,Pascal Benezech Dominique Pinon Marie-Laure Do...
2,1,6016,5.0,Alexandre Rodrigues Leandro Firmino Phellipe H...,Kátia Lund,photographer slum gang brazil,City of God (Cidade de Deus) (2002),Action Adventure Crime Drama Thriller,Alexandre Rodrigues Leandro Firmino Phellipe H...
3,1,7323,3.5,Daniel Brühl Katrin Saß Chulpan Khamatova Mari...,Bernd Lichtenberg,coma german democratic republic capitalism pol...,"Good bye, Lenin! (2003)",Comedy Drama,Daniel Brühl Katrin Saß Chulpan Khamatova Mari...
4,1,4973,4.5,Audrey Tautou Mathieu Kassovitz Rufus Lorella ...,Guillaume Laurant,female protagonist paris france france montmar...,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Comedy Romance,Audrey Tautou Mathieu Kassovitz Rufus Lorella ...


In [48]:
# View the result of the first five rows in the first merged chunks
useful_test.head()

,userId,movieId,title_cast,director,plot_keywords,title,genres,key_words
0,1,2011,,,,Back to the Future Part II (1989),Adventure Comedy Sci-Fi,Adventure Comedy Sci-Fi
1,1,4144,Maggie Cheung Tony Chiu-Wai Leung Ping Lam Siu...,Kar-Wai Wong,adultery loneliness neighbor infidelity,In the Mood For Love (Fa yeung nin wa) (2000),Drama Romance,Maggie Cheung Tony Chiu-Wai Leung Ping Lam Siu...
2,1,5767,,,,Teddy Bear (Mis) (1981),Comedy Crime,Comedy Crime
3,1,6711,Scarlett Johansson Bill Murray Akiko Takeshita...,Sofia Coppola,older man younger woman relationship lonelines...,Lost in Translation (2003),Comedy Drama Romance,Scarlett Johansson Bill Murray Akiko Takeshita...
4,1,7318,Jim Caviezel Maia Morgenstern Christo Jivkov F...,Benedict Fitzgerald,suffering torture brutality whipping,"Passion of the Christ, The (2004)",Drama,Jim Caviezel Maia Morgenstern Christo Jivkov F...


#### Drop columns not needed


In [49]:
# Drop unwanted colunms in the train data set
useful_train.drop(columns= ['title_cast','director','plot_keywords', 'genres'], inplace=True)
useful_train.head()

,userId,movieId,rating,title,key_words
0,1,3949,5.0,Requiem for a Dream (2000),Ellen Burstyn Jared Leto Jennifer Connelly Mar...
1,1,1175,3.5,Delicatessen (1991),Pascal Benezech Dominique Pinon Marie-Laure Do...
2,1,6016,5.0,City of God (Cidade de Deus) (2002),Alexandre Rodrigues Leandro Firmino Phellipe H...
3,1,7323,3.5,"Good bye, Lenin! (2003)",Daniel Brühl Katrin Saß Chulpan Khamatova Mari...
4,1,4973,4.5,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Audrey Tautou Mathieu Kassovitz Rufus Lorella ...


In [50]:
# Drop unwanted colunms in the test data set
useful_test.drop(columns= ['title_cast','director','plot_keywords', 'genres'], inplace=True)
useful_test.head()

,userId,movieId,title,key_words
0,1,2011,Back to the Future Part II (1989),Adventure Comedy Sci-Fi
1,1,4144,In the Mood For Love (Fa yeung nin wa) (2000),Maggie Cheung Tony Chiu-Wai Leung Ping Lam Siu...
2,1,5767,Teddy Bear (Mis) (1981),Comedy Crime
3,1,6711,Lost in Translation (2003),Scarlett Johansson Bill Murray Akiko Takeshita...
4,1,7318,"Passion of the Christ, The (2004)",Jim Caviezel Maia Morgenstern Christo Jivkov F...


In [2]:
from sklearn import preprocessing

**TfidfVectorizer**
We now need a mechanism to convert these textual features into a format which enables us to compute their relative similarities to one another.
This will allow us to translate our string-based collection of title_cast, director, plot_keywords, genres, key_words into numerical vectors to achieve this, we make use of **TfidfVectorizer**.

In [2]:
frame = []
for i in range(1624):
    t0 = time()
    le = preprocessing.LabelEncoder()
    train_dir = './data/chunked_train_data/merge_chunk_{0}.csv'.format(i + 1)
    test_dir = './data/chunked_test_data/test_merge_chunk_{0}.csv'.format(i + 1)
    loaded_data = pd.read_csv(train_dir)
    y = loaded_data['rating'].to_list()
    y = le.fit_transform(y)
    tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2),min_df=0, stop_words='english')
    X_vector = tf.fit_transform(loaded_data['key_words'].values.astype(str))
    model_df = pd.DataFrame(data=X_vector.toarray())
    model_df['userId'] = loaded_data['userId']
    logreg_model = MultinomialNB()
    logreg_model.fit(model_df, y)
    
    df_test = pd.read_csv(test_dir)
    X_count_test = tf.transform(df_test['key_words'].values.astype(str))
    text_df = pd.DataFrame(data=X_count_test.toarray())
    text_df['userId'] = df_test['userId']
    y_pred_logistic = logreg_model.predict(text_df)
    y_pred_logistic = le.inverse_transform(y_pred_logistic)
    df = pd.DataFrame({
        'Id': df_test['userId'],
        'rating': y_pred_logistic
    })
    
    t1 = time()
    print('Pridiction for chunk no: ', (i+1))
    print('Time elapsed: ', (t1-t0))
    # df.to_csv('file_001.csv',index=False)
    frame.append(df)

Pridiction for chunk no:  1
Time elapsed:  12.82479214668274
Pridiction for chunk no:  2
Time elapsed:  12.46979308128357
Pridiction for chunk no:  3
Time elapsed:  9.263437986373901
Pridiction for chunk no:  4
Time elapsed:  8.244291067123413
Pridiction for chunk no:  5
Time elapsed:  12.896457195281982
Pridiction for chunk no:  6
Time elapsed:  18.594558238983154
Pridiction for chunk no:  7
Time elapsed:  12.023573398590088
Pridiction for chunk no:  8
Time elapsed:  10.898711919784546
Pridiction for chunk no:  9
Time elapsed:  24.986931800842285
Pridiction for chunk no:  10
Time elapsed:  19.403557062149048
Pridiction for chunk no:  11
Time elapsed:  8.425351858139038
Pridiction for chunk no:  12
Time elapsed:  9.517669916152954
Pridiction for chunk no:  13
Time elapsed:  14.165422201156616
Pridiction for chunk no:  14
Time elapsed:  12.553562879562378
Pridiction for chunk no:  15
Time elapsed:  12.934664011001587
Pridiction for chunk no:  16
Time elapsed:  10.009536981582642
Pridict

Pridiction for chunk no:  132
Time elapsed:  15.478833198547363
Pridiction for chunk no:  133
Time elapsed:  11.752509832382202
Pridiction for chunk no:  134
Time elapsed:  15.262241840362549
Pridiction for chunk no:  135
Time elapsed:  12.65993595123291
Pridiction for chunk no:  136
Time elapsed:  12.494961261749268
Pridiction for chunk no:  137
Time elapsed:  12.470725774765015
Pridiction for chunk no:  138
Time elapsed:  9.842715740203857
Pridiction for chunk no:  139
Time elapsed:  9.5885329246521
Pridiction for chunk no:  140
Time elapsed:  11.589096069335938
Pridiction for chunk no:  141
Time elapsed:  13.422338962554932
Pridiction for chunk no:  142
Time elapsed:  17.133445024490356
Pridiction for chunk no:  143
Time elapsed:  10.837812900543213
Pridiction for chunk no:  144
Time elapsed:  14.01405382156372
Pridiction for chunk no:  145
Time elapsed:  23.37643575668335
Pridiction for chunk no:  146
Time elapsed:  22.054767847061157
Pridiction for chunk no:  147
Time elapsed:  11

Pridiction for chunk no:  261
Time elapsed:  11.58523178100586
Pridiction for chunk no:  262
Time elapsed:  10.513382911682129
Pridiction for chunk no:  263
Time elapsed:  12.885294914245605
Pridiction for chunk no:  264
Time elapsed:  16.686965942382812
Pridiction for chunk no:  265
Time elapsed:  21.798451900482178
Pridiction for chunk no:  266
Time elapsed:  13.58719801902771
Pridiction for chunk no:  267
Time elapsed:  16.352999925613403
Pridiction for chunk no:  268
Time elapsed:  11.982874155044556
Pridiction for chunk no:  269
Time elapsed:  12.6925208568573
Pridiction for chunk no:  270
Time elapsed:  12.115884065628052
Pridiction for chunk no:  271
Time elapsed:  13.717593908309937
Pridiction for chunk no:  272
Time elapsed:  13.444055080413818
Pridiction for chunk no:  273
Time elapsed:  16.66102385520935
Pridiction for chunk no:  274
Time elapsed:  18.280047178268433
Pridiction for chunk no:  275
Time elapsed:  16.231693983078003
Pridiction for chunk no:  276
Time elapsed:  

Pridiction for chunk no:  390
Time elapsed:  17.23668599128723
Pridiction for chunk no:  391
Time elapsed:  11.424787998199463
Pridiction for chunk no:  392
Time elapsed:  13.801800966262817
Pridiction for chunk no:  393
Time elapsed:  10.203809976577759
Pridiction for chunk no:  394
Time elapsed:  14.71923303604126
Pridiction for chunk no:  395
Time elapsed:  12.740109920501709
Pridiction for chunk no:  396
Time elapsed:  7.8317060470581055
Pridiction for chunk no:  397
Time elapsed:  33.33714509010315
Pridiction for chunk no:  398
Time elapsed:  12.800238132476807
Pridiction for chunk no:  399
Time elapsed:  30.248648166656494
Pridiction for chunk no:  400
Time elapsed:  13.060483932495117
Pridiction for chunk no:  401
Time elapsed:  9.863224029541016
Pridiction for chunk no:  402
Time elapsed:  12.510883092880249
Pridiction for chunk no:  403
Time elapsed:  20.70451307296753
Pridiction for chunk no:  404
Time elapsed:  9.643390893936157
Pridiction for chunk no:  405
Time elapsed:  1

Pridiction for chunk no:  519
Time elapsed:  11.196953058242798
Pridiction for chunk no:  520
Time elapsed:  9.021084070205688
Pridiction for chunk no:  521
Time elapsed:  8.830395936965942
Pridiction for chunk no:  522
Time elapsed:  19.28048014640808
Pridiction for chunk no:  523
Time elapsed:  19.907789945602417
Pridiction for chunk no:  524
Time elapsed:  19.390249013900757
Pridiction for chunk no:  525
Time elapsed:  12.09967589378357
Pridiction for chunk no:  526
Time elapsed:  7.639142751693726
Pridiction for chunk no:  527
Time elapsed:  15.73498821258545
Pridiction for chunk no:  528
Time elapsed:  12.079914093017578
Pridiction for chunk no:  529
Time elapsed:  13.98786997795105
Pridiction for chunk no:  530
Time elapsed:  18.432788848876953
Pridiction for chunk no:  531
Time elapsed:  18.767403841018677
Pridiction for chunk no:  532
Time elapsed:  10.629729986190796
Pridiction for chunk no:  533
Time elapsed:  13.309611082077026
Pridiction for chunk no:  534
Time elapsed:  13

...

Pridiction for chunk no:  1035
Time elapsed:  28.656289100646973
Pridiction for chunk no:  1036
Time elapsed:  15.800110816955566
Pridiction for chunk no:  1037
Time elapsed:  11.101829290390015
Pridiction for chunk no:  1038
Time elapsed:  26.579029083251953
Pridiction for chunk no:  1039
Time elapsed:  11.648091077804565
Pridiction for chunk no:  1040
Time elapsed:  13.253050804138184
Pridiction for chunk no:  1041
Time elapsed:  12.354716062545776
Pridiction for chunk no:  1042
Time elapsed:  13.880564212799072
Pridiction for chunk no:  1043
Time elapsed:  11.779166221618652
Pridiction for chunk no:  1044
Time elapsed:  16.673760890960693
Pridiction for chunk no:  1045
Time elapsed:  7.7661519050598145
Pridiction for chunk no:  1046
Time elapsed:  18.499711990356445
Pridiction for chunk no:  1047
Time elapsed:  13.434986114501953
Pridiction for chunk no:  1048
Time elapsed:  14.17721700668335
Pridiction for chunk no:  1049
Time elapsed:  15.307010889053345
Pridiction for chunk no:  

Pridiction for chunk no:  1162
Time elapsed:  27.22324824333191
Pridiction for chunk no:  1163
Time elapsed:  16.14987325668335
Pridiction for chunk no:  1164
Time elapsed:  24.604796171188354
Pridiction for chunk no:  1165
Time elapsed:  19.389613151550293
Pridiction for chunk no:  1166
Time elapsed:  12.333947896957397
Pridiction for chunk no:  1167
Time elapsed:  9.502793073654175
Pridiction for chunk no:  1168
Time elapsed:  19.836472749710083
Pridiction for chunk no:  1169
Time elapsed:  18.751428842544556
Pridiction for chunk no:  1170
Time elapsed:  17.537902116775513
Pridiction for chunk no:  1171
Time elapsed:  32.56283521652222
Pridiction for chunk no:  1172
Time elapsed:  14.700618028640747
Pridiction for chunk no:  1173
Time elapsed:  12.60579776763916
Pridiction for chunk no:  1174
Time elapsed:  15.199795961380005
Pridiction for chunk no:  1175
Time elapsed:  23.857933044433594
Pridiction for chunk no:  1176
Time elapsed:  22.151074171066284
Pridiction for chunk no:  1177

Pridiction for chunk no:  1289
Time elapsed:  10.104259967803955
Pridiction for chunk no:  1290
Time elapsed:  17.67124581336975
Pridiction for chunk no:  1291
Time elapsed:  12.716039180755615
Pridiction for chunk no:  1292
Time elapsed:  8.2677321434021
Pridiction for chunk no:  1293
Time elapsed:  26.297910690307617
Pridiction for chunk no:  1294
Time elapsed:  8.200645923614502
Pridiction for chunk no:  1295
Time elapsed:  20.36978578567505
Pridiction for chunk no:  1296
Time elapsed:  28.935688257217407
Pridiction for chunk no:  1297
Time elapsed:  13.13048005104065
Pridiction for chunk no:  1298
Time elapsed:  12.1244957447052
Pridiction for chunk no:  1299
Time elapsed:  13.953320026397705
Pridiction for chunk no:  1300
Time elapsed:  22.482980012893677
Pridiction for chunk no:  1301
Time elapsed:  16.594343900680542
Pridiction for chunk no:  1302
Time elapsed:  35.40938591957092
Pridiction for chunk no:  1303
Time elapsed:  14.209697008132935
Pridiction for chunk no:  1304
Time

Pridiction for chunk no:  1416
Time elapsed:  14.66907286643982
Pridiction for chunk no:  1417
Time elapsed:  19.791189908981323
Pridiction for chunk no:  1418
Time elapsed:  25.44045615196228
Pridiction for chunk no:  1419
Time elapsed:  14.899496078491211
Pridiction for chunk no:  1420
Time elapsed:  11.114818811416626
Pridiction for chunk no:  1421
Time elapsed:  32.20438599586487
Pridiction for chunk no:  1422
Time elapsed:  33.852404832839966
Pridiction for chunk no:  1423
Time elapsed:  11.983910083770752
Pridiction for chunk no:  1424
Time elapsed:  14.067739963531494
Pridiction for chunk no:  1425
Time elapsed:  8.381144046783447
Pridiction for chunk no:  1426
Time elapsed:  20.441048622131348
Pridiction for chunk no:  1427
Time elapsed:  25.55606508255005
Pridiction for chunk no:  1428
Time elapsed:  13.283005952835083
Pridiction for chunk no:  1429
Time elapsed:  33.735376834869385
Pridiction for chunk no:  1430
Time elapsed:  13.14125680923462
Pridiction for chunk no:  1431


Pridiction for chunk no:  1543
Time elapsed:  12.146461009979248
Pridiction for chunk no:  1544
Time elapsed:  16.956799030303955
Pridiction for chunk no:  1545
Time elapsed:  15.3101487159729
Pridiction for chunk no:  1546
Time elapsed:  16.294312000274658
Pridiction for chunk no:  1547
Time elapsed:  17.280625820159912
Pridiction for chunk no:  1548
Time elapsed:  12.605935096740723
Pridiction for chunk no:  1549
Time elapsed:  12.281148910522461
Pridiction for chunk no:  1550
Time elapsed:  22.843207120895386
Pridiction for chunk no:  1551
Time elapsed:  14.111646890640259
Pridiction for chunk no:  1552
Time elapsed:  9.294071912765503
Pridiction for chunk no:  1553
Time elapsed:  10.561065196990967
Pridiction for chunk no:  1554
Time elapsed:  11.307594060897827
Pridiction for chunk no:  1555
Time elapsed:  13.47666311264038
Pridiction for chunk no:  1556
Time elapsed:  17.052300930023193
Pridiction for chunk no:  1557
Time elapsed:  19.250838041305542
Pridiction for chunk no:  155

In [ ]:
tf = TfidfVectorizer(analyzer='word',  stop_words='english')
X_vector = tf.fit_transform(loaded_data['key_words'].values.astype(str))

In [4]:
# Save the merge table for the train dataset
for index, data in enumerate(frame):
    directory = './data/prediction_chunks/predict_{0}.csv'.format(index +1)
    data.to_csv(directory,index=False)

In [6]:
result = pd.concat(frame)
result.shape


(5000019, 2)

In [47]:
result['movieId'] = test['movieId'].tolist()
result.head()

,Id,rating,movieId
0,1,4.0,2011
1,1,4.0,4144
2,1,4.0,5767
3,1,4.0,6711
4,1,3.0,7318


In [48]:
test['movieId'].nunique()

39643

In [58]:
work = result.copy()
work['Id'] = work['Id'].values.astype(str)
work['movieId'] = work['movieId'].values.astype(str)
work['Id'] = (pd.Series(work[['Id', 'movieId']].values.tolist()).str.join('_')).tolist()
work.head()

,Id,rating,movieId
0,1_2011,4.0,2011
1,1_4144,4.0,4144
2,1_5767,4.0,5767
3,1_6711,4.0,6711
4,1_7318,3.0,7318


In [59]:
work.shape

(5000019, 3)

In [60]:
work.nunique()

Id         5000019
rating           6
movieId      39643
dtype: int64

In [61]:
result.nunique()

Id         162350
rating          6
movieId     39643
dtype: int64

In [62]:
work.drop(columns=['movieId'], inplace = True)
work.head()

,Id,rating
0,1_2011,4.0
1,1_4144,4.0
2,1_5767,4.0
3,1_6711,4.0
4,1_7318,3.0


In [63]:
work = work.drop_duplicates(subset=['Id'])
work.head()

,Id,rating
0,1_2011,4.0
1,1_4144,4.0
2,1_5767,4.0
3,1_6711,4.0
4,1_7318,3.0


In [64]:
work.shape

(5000019, 2)

In [65]:
work.to_csv('file_001.csv',index=False)

In [ ]:
model_df = pd.DataFrame(data=X_vector.toarray())
model_df.head()

In [ ]:
# Create first 100 consine similarities 
cosine_sim_list = []

real_index = 0
stop_count = 300
real_vector_list = vector_list[real_index:stop_count]


for index, tf_matrix in enumerate(real_vector_list):
    cosine_sim = "cosine_sim_{0}".format(real_index + 1)
    
    globals()[cosine_sim] = cosine_similarity(globals()[tf_matrix], 
                                        globals()[tf_matrix])
    cosine_sim_list.append(cosine_sim)
    
    real_index = real_index + 1
    if real_index == stop_count :
        break

print (cosine_sim_1.shape)

In [62]:
# Print cosine similarity value for the 200th chunk
cosine_sim_100.shape

(6094, 6094)

In [63]:
# View the next start point
real_index

100

In [64]:
# Create next 200 consine similarities 

stop_count = 150
real_vector_list = vector_list[real_index:stop_count]


for index, tf_matrix in enumerate(real_vector_list):
    cosine_sim = "cosine_sim_{0}".format(real_index + 1)
    
    globals()[cosine_sim] = cosine_similarity(globals()[tf_matrix], 
                                        globals()[tf_matrix])
    cosine_sim_list.append(cosine_sim)
    
    real_index = real_index + 1
    if real_index == stop_count :
        break

# View the last cosine similarity value generated 
print (cosine_sim_150.shape)


(6023, 6023)


In [65]:
# View the next start point
real_index

150

In [66]:
# Create next 200 consine similarities 

stop_count = 200
real_vector_list = vector_list[real_index:stop_count]


for index, tf_matrix in enumerate(real_vector_list):
    cosine_sim = "cosine_sim_{0}".format(real_index + 1)
    
    globals()[cosine_sim] = cosine_similarity(globals()[tf_matrix], 
                                        globals()[tf_matrix])
    cosine_sim_list.append(cosine_sim)
    
    real_index = real_index + 1
    if real_index == stop_count :
        break

# View the last cosine similarity value generated 
print (cosine_sim_200.shape)

(5638, 5638)


In [67]:
# View the next start point
real_index

200

In [68]:
# Create next 200 consine similarities 

stop_count = 250
real_vector_list = vector_list[real_index:stop_count]


for index, tf_matrix in enumerate(real_vector_list):
    cosine_sim = "cosine_sim_{0}".format(real_index + 1)
    
    globals()[cosine_sim] = cosine_similarity(globals()[tf_matrix], 
                                        globals()[tf_matrix])
    cosine_sim_list.append(cosine_sim)
    
    real_index = real_index + 1
    if real_index == stop_count :
        break

# View the last cosine similarity value generated 
print (cosine_sim_250.shape)

(5638, 5638)


In [70]:
# View the next start point
real_index

250

In [ ]:
# Create next 200 consine similarities 

stop_count = 300
real_vector_list = vector_list[real_index:stop_count]


for index, tf_matrix in enumerate(real_vector_list):
    cosine_sim = "cosine_sim_{0}".format(real_index + 1)
    
    globals()[cosine_sim] = cosine_similarity(globals()[tf_matrix], 
                                        globals()[tf_matrix])
    cosine_sim_list.append(cosine_sim)
    
    real_index = real_index + 1
    if real_index == stop_count :
        break

# View the last cosine similarity value generated 
print (cosine_sim_300.shape)

#### Spliting of dataset
We split our transformed dataset into 50 part, converting same into dataframe and save to local storage. This process
gives 199957 rows for the first 49 chunks and 199952 rows for the last chunk

In [ ]:
def create_chunk_list(obj, limit):
    chunk = []
    chunks = []
    obj_len = len(obj)
    
    for index, value in enumerate(obj) :
        chunk.append(value)
        if ( (len(chunk) == limit ) | ((index +1) == obj_len )) :
            chunks.append(chunk)
            chunk = []
    
    return chunks
            

####  Convert vectorised data set back to dataframe form

In [ ]:
vectoried_df = pd.DataFrame(data=tf_authTags_matrix.toarray(),columns = vector.get_feature_names())
vectoried_df.head()

We now can compute the similarity between each vector within our matrix. This is done by making use of the `cosine_similarity` function provided to us by `sklearn`.

In [ ]:
cosine_sim_authTags = cosine_similarity(tf_authTags_matrix, 
                                        tf_authTags_matrix)
print (cosine_sim_authTags.shape)

### 4.2 Collaborative filtering

### 4.3 Rating Prediction

As motivated previously, in some cases we may wish to directly calculate what rating a user _would_ give a book that they haven't read yet. 

We can modify our content-based filtering algorithm to do this in the following manner: 

   1. Select a reference user from the database and a reference item (movie) they have _not_ rated. 
   2. For the user, gather the similarity values between the reference item and each item the user _has_ rated. 
   3. Sort the gathered similarity values in descending order. 
   4. Select the $k$ highest similarity values which are above a given threshold value, creating a collection $K$. 
   5. Compute a weighted average rating from these values, which is the sum of the similarity values of each item multiplied by its assigned user-rating, divided by the sum of the similarity values. This can be expressed in formula as:
   
   $$ \hat{R}_{ju} = \frac{\sum_{i \in K} s_{ij} \times r_{iu}}{\sum_{i \in K} s_{ij}}   $$
   
   where $\hat{R}_{ju}$ is the weighted average computed for the reference item $j$ and reference user $u$, $K$ is the collection of items, $s_{ij}$ is the similarity computed between items $i$ and $j$, and $r_{iu}$ is the known rating user $u$ has given item $i$.
   6. We return the weighted average $\hat{R}_{ju}$ as the prediction for our reference item.
   
   
We implement this algorithmic process in the function below:

<a id="five"></a>
## 5. Modelling
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Modelling ⚡ |
| :--------------------------- |
| In this section, you are required to create one or more regression models that are able to accurately predict the thee hour load shortfall. |

---

<a id="six"></a>
## 6. Model Performance
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model performance ⚡ |
| :--------------------------- |
| In this section you are required to compare the relative performance of the various trained ML models on a holdout dataset and comment on what model is the best and why. |

---

In [ ]:
# Compare model performance

In [ ]:
# Choose best model and motivate why it is the best choice

<a id="seven"></a>
## 7. Model Explanations
<a class="anchor" id="1.1"></a>
<a href=#cont>Back to Table of Contents</a>

---
    
| ⚡ Description: Model explanation ⚡ |
| :--------------------------- |
| In this section, you are required to discuss how the best performing model works in a simple way so that both technical and non-technical stakeholders can grasp the intuition behind the model's inner workings. |

---

In [ ]:
# discuss chosen methods logic

Run the next cell to make sure the experiment as ended. It notifies comit.

In [ ]:
experiment.end()